In [11]:
# grp_opts.py
# group options based on right, symbol and strike.
# this makes it easy to delete unwanted ones on inspection.
import pandas as pd

def grp_opts(df):
    '''Groups options and sorts strikes by puts and calls
    Arg: 
       df as dataframe. Requires 'symbol', 'strike' and 'dte' fields in the df
    Returns: sorted dataframe'''
    
    gb = df.groupby('right')

    if 'C' in [k for k in gb.indices]:
        df_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, False, True])
    else:
        df_calls =  pd.DataFrame([])

    if 'P' in [k for k in gb.indices]:
        df_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'dte', 'strike'], ascending=[True, False, False])
    else:
        df_puts =  pd.DataFrame([])

    df = pd.concat([df_puts, df_calls]).reset_index(drop=True)
    
    return df

In [29]:
from os import listdir
import pandas as pd
import sys

sys.path.append('./py')

from grp_opts import grp_opts

fspath = '../data/snp/' # path for pickles

minOptPrice = 0.1
minRom = 0.85

fs = listdir(fspath)

cols = ['optId', 'symbol', 'right', 'expiration', 'dte', 'strike', 'undPrice', 
'lo52', 'hi52', 'Fall', 'Rise', 'loFall', 'hiRise', 'std3', 'loStd3', 'hiStd3', 
'lotsize', 'optPrice', 'optMargin', 'rom']

optsList = [f for f in fs if f[-3:] == 'pkl']

df1 = pd.concat([pd.read_pickle(fspath+f) for f in optsList], axis=0, sort=True).reset_index(drop=True)[cols]

df2 = df1[((df1.strike > df1.hi52) | (df1.strike < df1.lo52)) & (df1.optPrice > minOptPrice) & (df1.rom > minRom)]

df2 = grp_opts(df2)

In [30]:
df2

,optId,symbol,right,expiration,dte,strike,undPrice,lo52,hi52,Fall,Rise,loFall,hiRise,std3,loStd3,hiStd3,lotsize,optPrice,optMargin,rom
0,360773420.0,BKNG,P,20190426,10.0,1605.0,1846.23,1606.27,2228.99,293.50,268.41,1552.73,2114.64,100.885788,1745.344212,1947.115788,1.0,1.70,21.64,1.979667
1,360773417.0,BKNG,P,20190426,10.0,1602.5,1846.23,1606.27,2228.99,293.50,268.41,1552.73,2114.64,100.885788,1745.344212,1947.115788,1.0,1.70,21.61,1.982416
2,356373262.0,BKNG,P,20190426,10.0,1600.0,1846.23,1606.27,2228.99,293.50,268.41,1552.73,2114.64,100.885788,1745.344212,1947.115788,1.0,1.65,21.45,1.938462
3,360773414.0,BKNG,P,20190426,10.0,1597.5,1846.23,1606.27,2228.99,293.50,268.41,1552.73,2114.64,100.885788,1745.344212,1947.115788,1.0,1.65,21.58,1.926784
4,356373259.0,BKNG,P,20190426,10.0,1590.0,1846.23,1606.27,2228.99,293.50,268.41,1552.73,2114.64,100.885788,1745.344212,1947.115788,1.0,1.60,21.62,1.864940
5,360773390.0,BKNG,P,20190426,10.0,1577.5,1846.23,1606.27,2228.99,293.50,268.41,1552.73,2114.64,100.885788,1745.344212,1947.115788,1.0,1.55,21.88,1.785192
6,360773375.0,BKNG,P,20190426,10.0,1572.5,1846.23,1606.27,2228.99,293.50,268.41,1552.73,2114.64,100.885788,1745.344212,1947.115788,1.0,1.50,22.05,1.714286
7,356373247.0,BKNG,P,20190426,10.0,1570.0,1846.23,1606.27,2228.99,293.50,268.41,1552.73,2114.64,100.885788,1745.344212,1947.115788,1.0,1.50,21.93,1.723666
8,360773360.0,BKNG,P,20190426,10.0,1565.0,1846.23,1606.27,2228.99,293.50,268.41,1552.73,2114.64,100.885788,1745.344212,1947.115788,1.0,1.45,22.33,1.636364
9,360773348.0,BKNG,P,20190426,10.0,1562.5,1846.23,1606.27,2228.99,293.50,268.41,1552.73,2114.64,100.885788,1745.344212,1947.115788,1.0,1.45,22.42,1.629795
